In [ ]:
# Imports
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.providers.aer import AerSimulator
from qiskit.visualization import plot_histogram

sim = AerSimulator()  # Initialize a simulator instance

In [ ]:
# -- Demonstrating Quantum Communication using Entanglement --
# First, setup the channels and encoding scheme for a two-qubit message transfer

message = "01"  # some message we want to send from Alice to Bob
alice = QuantumCircuit(2, 2)  # "Alice", represented by a two-qubit quantum circuit
bob = QuantumCircuit(2, 2)  # "Bob", represented by another two-qubit quantum circuit

def encodeMessage(someQC, someMessage):  # encodes a message in a quantum circuit with X gates
    for i in range(1, len(message)+1):  # indices go 1, 2, 3, 4, ...
        if someMessage[-i] == "1":  # negative indices = counted from the end of the string: think bit place values
            someQC.x(i-1)  # X gate the correct qubit if the message input is 1 (but these indices start at 0 so subtract 1)
    return someQC

def measureMessage(someQC, messageLen):  # measures a message of any length stored in qubits to the classical register
    someQC.measure(list(range(messageLen)), list(range(messageLen)))  # measure bits [0, 1, ..., n]
    return someQC

In [ ]:
# Alice must first encode the message
alice = encodeMessage(alice, message)
# And on his end, Bob will measure the message
bob = measureMessage(bob, len(message))

alice.draw(output="mpl");

bob.draw(output="mpl");

# If we do nothing and just run this now...
commResults = sim.run(alice.compose(bob)).result().get_counts()
plot_histogram(commResults);
# The message Bob measures is exactly the one Alice encoded — nothing happens, as expected

In [ ]:
# With entanglement, it gets a bit more complex...
message = "00"

alice = QuantumCircuit(2, 2)  # create and encode, same as before
alice = encodeMessage(alice, message)
# Now Alice entangles her qubits using H and CNOT
alice.h(1)  # put qubit 1 in superposition
alice.cx(1, 0)  # CNOT with the superposed qubit as control

bob = QuantumCircuit(2, 2)
# And for Bob to receive the message, he must undo the entanglement and then perform measurement
bob.cx(0, 1)  # Opposite: CNOT with control and target swapped
bob.h(0)  # Again, the H is used on the other qubit
bob = measureMessage(bob, len(message))

alice.draw(output="mpl");
bob.draw(output="mpl");

# This is still the same, though: we are only adding an extra step somewhere.
# To figure out how we can get some quantum advantage, let's look at the possible entangled states Alice could have
# depending on what the message is.

In [ ]:
stateVectors = []
# Experimenting with messages
for message in ["00", "01", "10", "11"]:
    alice = QuantumCircuit(2, 2)
    alice = encodeMessage(alice, message)
    alice.h(1)
    alice.cx(1, 0)
    print(f"Statevector for message {message}:")
    stateVectors.append(Statevector(alice).draw("latex"))

In [ ]:
print("Statevector for message 00:")
stateVectors[0]
# 00  01  10  11 

In [ ]:
print("Statevector for message 01:")
stateVectors[1]
# 00  01  10  11

In [ ]:
print("Statevector for message 10:")
stateVectors[2]
# 00  01  10  11

In [ ]:
print("Statevector for message 11:")
stateVectors[3]
# 00  01  10  11

We see that all the states can be reached from one another through X and Z gates.
So what if we entangle the qubits BEFORE we encode the message?
Then we can apply X and Z to one of the qubits, and the other will reflect that as well...
This means that Alice can send the second qubit to Bob before even encoding the message!
If we take this a step further, it means that a hypothetical third person could just churn out
entangled pairs, say in the Bell state, and keep feeding one to Alice and one to Bob, and Alice
would just have to apply X and Z gates to hers, upon which Bob would instantly get the message upon measurement!
We are transmitting **two** classical bits of information by manipulating only **one** qubit...
and as is often the case, this scales up **exponentially** the more qubits we have!

In [ ]:
# Implementing Communication

message = "11"

charlie = QuantumCircuit(2, 2)  # Third party whose only job is to generate entangled pairs
charlie.h(1)
charlie.cx(1, 0)  # standard procedure to create a Bell state

# We imagine that Charlie now sends qubit 0 to Bob, and qubit 1 to Alice

alice = QuantumCircuit(2, 2)  # Alice will only perform operations on qubit 1
if message[-2] == "1":
    # If the left bit of the message is 1, we use an X gate
    # Note that due to entanglement, an X or Z gate on one qubit forces a change in the other's state
    alice.x(1)
if message[-1] == "1":
    # If the right bit is 1, we use a Z gate (see state vectors from above)
    alice.z(1)

# Now imagine that Alice sends qubit 1 to Bob (only sending 1 qubit of information!)

# Then Bob can measure using his standard procedure and get the same output message
bob = QuantumCircuit(2, 2)
bob.cx(0, 1)
bob.h(0)  # Undo the entanglement, same way as before — but now the output will be different
bob = measureMessage(bob, 2)

plot_histogram(sim.run(charlie.compose(alice.compose(bob))).result().get_counts());

# And we see that whatever the message is, Bob measures exactly that 100% of the time!
